# Human Detection Project

In [1]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from sklearn import svm

# Lectura de imagenes

In [3]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

In [4]:
path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain = leerFotos(path)

path = 'C:\\Users\\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

In [96]:
def generaImagenesRecortadas(dataPos,dataNeg):    
    numeroCortesxPh = 15      # Numero de recortadas aleatorias por foto
    rowsFotoPersona = dataPos[0].shape[0]
    colsFotoPersona = dataPos[0].shape[1]
    
    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        rowsFotoEntorno = dataNeg[i].shape[0]
        colsFotoEntorno = dataNeg[i].shape[1]
        dimCol = colsFotoEntorno - colsFotoPersona
        dimRow = rowsFotoEntorno - rowsFotoPersona
        
        for j in range(numeroCortesxPh):    
            randRow = random.randint(0,dimRow-1)
            randCol = random.randint(0,dimCol-1)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona ,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroCortesxPh*i] = randomImage
            
    return fotosNegTrain

In [97]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

# Extracción de Caracterísitcas

In [98]:
def vectolizarHOG(imagenes,esPositivo):
    ### Esata funcion aplica HOG a todas las imagenes del diccionario
    
    #tamaño del vector
    tamVector = len(hog(imagenes[0],transform_sqrt = True))

    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenes = np.zeros((len(imagenes),tamVector))
    
    for i in range(len(imagenes)):
        fdImagenes[i,:] = hog(imagenes[i],transform_sqrt = True)
    
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenes,y

In [99]:
fdPosTrain,yPos = vectolizarHOG(fotosPosTrain,1)

In [100]:
fdNegTrain,yNeg = vectolizarHOG(fotosNegTrain,0)

In [101]:
fotosTrain = np.vstack((fdPosTrain,fdNegTrain))
y = np.vstack((yPos,yNeg))

# Entrenamiento del modelo

In [105]:
clf = svm.LinearSVC()
clf.fit(fotosTrain,y.ravel())

In [103]:
clf.score(fdPosTrain,yPos)

1.0

# Slide Window

In [58]:
def secImagen(imagen,maskHeight,maskWidth,desp):
    arrayImagen = {}
    aux = 0
    for i in range(0,imagen.shape[0],desp):        
        for j in range(0,imagen.shape[1],desp):
            if (i+maskHeight < imagen.shape[0] and j+maskWidth < imagen.shape[1]):
                arrayImagen[aux] = imagen[i:i+maskHeight,j:j+maskWidth]
                aux = aux + 1
    return arrayImagen

In [59]:
path = 'C:\\Users\Jhonny Chicaiza\\Desktop\\Human_Detection_Project-master\\Test\\pos'
fotostest = leerFotos(path)

In [60]:
partImagen = secImagen(fotostest[6],134,70,10)

# Prueba del Modelo

In [106]:
listOk = []
for i in range(len(partImagen)):
    prueba = hog(partImagen[i],transform_sqrt = True);
    if (1 == clf.predict(prueba.reshape(1,-1))):
        listOk.append(i);

In [111]:
cv2.imshow('img ', fotostest[6])
cv2.imshow('xd ', partImagen[1978])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [107]:
print(listOk)

[113, 166, 306, 396, 433, 468, 501, 503, 507, 753, 765, 766, 786, 791, 798, 836, 837, 853, 856, 863, 868, 925, 926, 995, 1020, 1021, 1104, 1135, 1139, 1159, 1160, 1176, 1205, 1209, 1219, 1220, 1221, 1233, 1234, 1252, 1256, 1257, 1281, 1282, 1349, 1351, 1427, 1428, 1438, 1445, 1502, 1557, 1558, 1571, 1581, 1585, 1597, 1620, 1621, 1627, 1628, 1629, 1631, 1632, 1637, 1641, 1651, 1667, 1670, 1682, 1690, 1697, 1698, 1717, 1721, 1740, 1768, 1769, 1771, 1787, 1805, 1808, 1810, 1841, 1856, 1857, 1874, 1908, 1911, 1938, 1939, 1944, 1978, 1996, 2008, 2009, 2011, 2015, 2050, 2057, 2078, 2081, 2120, 2127, 2151, 2153, 2189, 2190, 2197, 2221, 2267, 2291, 2293, 2337, 2355, 2361, 2407, 2425, 2426, 2442, 2446, 2448, 2516, 2547, 2557, 2565, 2577, 2582, 2617, 2627, 2635, 2687, 2697]
